--> NER the bottle neck? --> Chat with Nawar?

--> A comparison between Semantic Scholar, Grobid, LaTexWalker. (provide pros & cons & feasibility of each approach & next step)

**TODO** --> Run this script on a bigger scale, see the success failure rate S3 pipeline

What I did:
manually randomly selected more paper --> processed --> found more issues (sometimes I don't know, sometimes I know '}' ) --> improve algorithm (including Regex) --> increase duarbility


v7 --> v8. added more PDFs

v8 --> v9 reduced errors from 6 down to 3

V10  --> addressing "/newblock" + more robust @ detection

resolved:

for example it would falsely match this line:

\g@addto@macro\@parboxrestore{\lineskiplimit\normallineskip}

or this line (false):

\providecommand \@ifxundefined [1]{\@ifx{#1\undefined}



| Style Detected         | What to Do                                         |
| ---------------------- | -------------------------------------------------- |
| `\bibitem`             | Already handled via `parse_bibitem_block()`        |
| `\bibliography{}`      | Locate and parse `.bib` files using `bibtexparser` |
| `\bibliographystyle{}` | Optional: record it (not used in parsing directly) |



# Batch 1:

## Not sure:
1912.00001


## poor format:
File: 1912.04804.tex


## failed expansion:
File: 1912.04801.tex


# Batch 2.  - June 13th



@ARTICLE{Zevin2019,
	author = {{Zevin}, M. and {Kremer}, K. and et al.},
     year = 2019,
}


basically a huge dump of info

why 1911.00018 works but 1911.00003 & 1911.00010 doesn't work?

**Ans:  Slaccitation -  missing a closing "}". How????**

**TODO**

1911.00008 looks like 1912.04804

  \bibfield  {author} {\bibinfo {author} {\bibfnamefont {A.}~\bibnamefont
  {Polkovnikov}}, \bibinfo {author} {\bibfnamefont {K.}~\bibnamefont


\bibliographystyle{hyperplain}
\begin{thebibliography}{10}


1911.00006 why ?

\bibitem{Agol11}
Ian Agol.
\newblock Ideal triangulations of pseudo-{A}nosov mapping tori.
\newblock In {\em Topology and Geometry in Dimension Three: Triangulations,
  Invariants, and Geometric Structures (Proceedings of the Jacofest
  conference)}, volume 560 of {\em AMS Contemporary Mathematics}, pages 1--18,
  2011,  \href{http://arxiv.org/abs/1008.1606v2}{{arXiv:1008.1606v2}}.


ASK Nawar --> NER
Attempt to find a dataset?


Example 1911.00003



> @article{Banks:1975gq,
	Author = {Banks, Tom and Susskind, Leonard and Kogut, John B.},
	Date-Added = {2019-04-08 14:22:07 +0200},
	Date-Modified = {2019-04-11 17:39:37 +0200},
	Doi = {10.1103/PhysRevD.13.1043},
	Journal = {Phys. Rev. D},
	Pages = {1043},
	Reportnumber = {CLNS-318},
	Slaccitation = {Title = {{Strong Coupling Calculations of Lattice Gauge Theories: (1+1)-Dimensional Exercises}},
	Volume = {13},
	Year = {1976},
	Bdsk-Url-1 = {http://dx.doi.org/10.1103/PhysRevD.13.1043}}



TODO:

Adding Href handling

Compare with Grobib

In [1]:
!rm -rf /content/parsed_references.txt

In [2]:
!rm -rf Latex-papers

In [3]:
!git clone https://github.com/Luke-Guan-FirstPrinciples/Latex-papers.git

Cloning into 'Latex-papers'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 63 (delta 6), reused 39 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 1.35 MiB | 5.78 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [4]:
!pip install bibtexparser pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=db50966894e903064e17666bbda973e633bc3d6356525e498c8848ff9312a461
  Stored in directory: /root/.cache/pip/wheels/16/fb/76/306387739cf9d53b1c39b0c8aadbbb17dc05f256756d8fd915
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=fb60d602e518643da0b24bfde6a41348fc2e91ba28febd6ad20491468a7d6626
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built bibtexparser pylatexenc


In [5]:
import os

In [6]:
!pwd

/content


In [7]:
import re
import bibtexparser
from pylatexenc.latexwalker import LatexWalker, LatexCharsNode, LatexMacroNode, LatexGroupNode, LatexEnvironmentNode


def clean_latex_with_latexwalker(latex_string):
    walker = LatexWalker(latex_string)
    try:
        nodelist, pos, _ = walker.get_latex_nodes(pos=0)
    except Exception as e:
        print(f"❌ LatexWalker failed: {e}")
        return latex_string

    def _extract_text(nodes):
        text = ''
        for node in nodes:
            if isinstance(node, LatexCharsNode):
                text += node.chars
            elif isinstance(node, LatexGroupNode):
                text += _extract_text(node.nodelist)
            elif isinstance(node, LatexMacroNode):
                if node.macroname == 'href' and len(node.nodeargs) == 2:
                    # \href{url}{text} → just keep {text}
                    text += _extract_text(node.nodeargs[1].nodelist)
                elif node.macroname in ['em', 'textbf', 'textit']:
                    if node.nodeargs:
                        text += _extract_text(node.nodeargs[0].nodelist)
                elif node.macroname == 'newblock':
                    # Treat as a sentence separator
                    text += '. '
                else:
                    # Unknown macro — ignore or log
                    pass
        return text

    result = re.sub(r'\s+', ' ', _extract_text(nodelist)).strip()
    return result


def parse_bibitem_block_robust(tex_content):
    print("🔍 DEBUG: Starting parse_bibitem_block_robust")

    # Trim everything outside thebibliography
    bib_section = re.search(r'\\begin\{thebibliography\}.*?\\end\{thebibliography\}', tex_content, flags=re.DOTALL)
    if not bib_section:
        print("❌ DEBUG: No thebibliography section found")
        return []

    content = bib_section.group(0)
    # print(f"✅ DEBUG: Found bibliography section, length: {len(content)}")
    # print(f"📝 DEBUG: First 200 chars of content: {repr(content[:200])}")

    # Use a more robust pattern to find all \bibitem entries
    # This pattern captures everything from \bibitem to the next \bibitem or \end{thebibliography}
    pattern = r'\\bibitem(?:\[[^\]]*\])?\{([^}]+)\}(.*?)(?=\\bibitem|\\end\{thebibliography\})'
    matches = re.findall(pattern, content, flags=re.DOTALL)

    # print(f"🔍 DEBUG: Found {len(matches)} regex matches")
    # for i, (key, body) in enumerate(matches):
    #     print(f"📋 DEBUG: Match {i+1}: key='{key}', body_length={len(body)}")
    #     print(f"📋 DEBUG: Match {i+1} body preview: {repr(body[:100])}")

    parsed = []
    for i, (key, body) in enumerate(matches):
        key = key.strip()
        body = body.strip()

        # print(f"🔧 DEBUG: Processing match {i+1}: key='{key}'")

        # # Skip if body is empty
        # if not body:
        #     print(f"⚠️ DEBUG: Skipping match {i+1} - empty body")
        #     continue

        # Clean the body - remove \newblock and other LaTeX commands
        # print(f"🧹 DEBUG: Cleaning body for '{key}'")
        cleaned = clean_latex_with_latexwalker(body)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()

        print(f"✨ DEBUG: Cleaned result for '{key}': {repr(cleaned[:100])}")

        if cleaned:  # Only add if we have actual content
            parsed.append({
                'key': key,
                'raw': cleaned
            })
            # print(f"✅ DEBUG: Added entry '{key}' to results")
        else:
            print(f"⚠️ DEBUG: Skipping '{key}' - cleaned content is empty")

    # print(f"🎉 DEBUG: Final result: {len(parsed)} entries parsed")
    return parsed


# V9
# ✅ Improvements Plan
# Use a manual parser fallback: When a BibTeX entry is incomplete, bibtexparser will likely fail. So:

# Detect and try to salvage broken entries by bracket-counting.

# Allow parsing of partial or malformed entries as best-effort, rather than discarding them entirely.

# More flexible regex: Match from @type{ up to the next @ or end of file, and try to balance braces manually to find complete entries.

# Warn and keep raw string if it can't parse.

# 🧠 Strategy: Balanced Braces Scanning
# We can do this by scanning line-by-line from each @...{ and tracking {/} balance, collecting until balance reaches zero (or EOF).

def extract_embedded_bibtex_entries(tex_content):
    print("extract_embedded_bibtex_entries")
    """
    Extract BibTeX entries from a LaTeX string robustly,
    even when entries are malformed or have missing closing brackets.
    """
    entries = []
    lines = tex_content.splitlines()
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.lower().startswith('@'):
            entry_lines = [line]
            brace_depth = line.count('{') - line.count('}')
            i += 1
            while i < len(lines) and brace_depth > 0:
                entry_lines.append(lines[i])
                brace_depth += lines[i].count('{') - lines[i].count('}')
                i += 1
            entry_text = '\n'.join(entry_lines).strip()

            # Quick cleanup: ensure it ends with }
            if not entry_text.endswith('}'):
                print(f"⚠️ Incomplete entry (missing closing brace):\n{entry_text[:100]}...\nTrying to recover...")

                # Try to auto-close it
                entry_text += '\n}'

            try:
                bib_database = bibtexparser.loads(entry_text)
                if bib_database.entries:
                    entries.extend(bib_database.entries)
                else:
                    print(f"⚠️ Could not parse entry: {entry_text[:100]}...")
            except Exception as e:
                print(f"❌ Failed to parse BibTeX entry:\n{entry_text[:100]}...\nError: {e}")
                # Optionally, store raw fallback
                entries.append({'ID': 'unknown', 'raw': entry_text})
        else:
            i += 1

    print(f"✅ Parsed {len(entries)} BibTeX entries (including fallbacks)")
    return entries

import re

def parse_latex_references(tex_path):
    print(f"📂 DEBUG: Opening file: {tex_path}")

    with open(tex_path, 'r', encoding='utf-8') as f:
        tex_content = f.read()


    # Original v9.     bibtex_entry_regex = re.compile(r'@\w+\s*{')

    # --- Robust BibTeX Entry Detection ---
    bibtex_entry_regex = re.compile(
        r'^[ \t]*@(?:article|book|inproceedings|misc|phdthesis|techreport|incollection)\s*\{',
        flags=re.IGNORECASE | re.MULTILINE
    )

    for lineno, line in enumerate(tex_content.splitlines(), start=1):
        if bibtex_entry_regex.match(line):
            print(f"🐛 DEBUG: BibTeX entry detected on line {lineno}: {line.strip()}")
            break  # Remove this break if you want to list all BibTeX matches

    if bibtex_entry_regex.search(tex_content):
        print("🔍 DEBUG: Found embedded BibTeX entries")
        return extract_embedded_bibtex_entries(tex_content)

    # --- thebibliography Environment Detection ---
    for lineno, line in enumerate(tex_content.splitlines(), start=1):
        if '\\begin{thebibliography}' in line:
            print(f"📚 DEBUG: Found \\begin{{thebibliography}} on line {lineno}: {line.strip()}")
            break

    if '\\begin{thebibliography}' in tex_content:
        print("🔍 DEBUG: Found thebibliography environment")
        return parse_bibitem_block_robust(tex_content)

    # --- No Recognized Format Found ---
    print("❌ DEBUG: No bibliography format detected")
    return []



In [ ]:
from pprint import pformat

# --- Batch Processing Script ---

if __name__ == '__main__':
    folder_path = '/content/Latex-papers/batch2'  # Folder containing .tex files
    output_path = '/content/parsed_references.txt'

    with open(output_path, 'w', encoding='utf-8') as out_file:
        for filename in os.listdir(folder_path):
            if filename.endswith('.tex'):
                full_path = os.path.join(folder_path, filename)
                print(f"\n📄 Processing: {filename}")
                try:
                    references = parse_latex_references(full_path)
                    out_file.write(f"File: {filename}\n")
                    out_file.write(pformat(references, indent=2))
                    out_file.write("\n\n" + "="*80 + "\n\n")
                except Exception as e:
                    error_msg = f"❌ Failed to parse {filename}: {e}"
                    print(error_msg)
                    out_file.write(f"{error_msg}\n\n")




📄 Processing: 1911.00016.tex
📂 DEBUG: Opening file: /content/Latex-papers/batch2/1911.00016.tex
🐛 DEBUG: BibTeX entry detected on line 1166: @ARTICLE{York2000AJ....120.1579Y,
🔍 DEBUG: Found embedded BibTeX entries
extract_embedded_bibtex_entries
❌ Failed to parse BibTeX entry:
@PHDTHESIS{1990PhDT.......263C,
   author = {{Carini}, M.~T.},
    title = "{The time scales of the ...
Error: 'january'


I asked Gemini:



Show more

Conversation with Gemini
Are the following references the same style:



    'raw': 'authorD.Acosta etal. (collaborationCDF Collaboration), '

           'journalPhys. Rev. Lett. volume95, pages022003 (year2005).'},





    'raw': 'Aldous, D.J. (1985). Exchangeability and related topics. In cole '

           "d't de Probabilits de Saint-Flour XIII—1983, pages 1198. "

           'Springer.'},



    'raw': 'Berg, A.R., Jordn, T.: Algorithms for graph rigidity and scene '

           'analysis. In: Proc. 11th European Symposium on Algorithms (ESA '

           "'03), LNCS, vol. 2832, pp. 7889. (2003)"},





    'raw': "A. D. Aleksandrov and V. V. Strel'tsov, The isoperimetric problem "

           'and estimates of the length of a curve on a surface, Proc. Steklov '

           'Inst. Math. 76 (1965), 67-80; English transi., Proc. Steklov Inst '

           'Math. (1967), 81-99.'},



    'raw': 'Coffman, V., Kundu, J., Wootters, W.K.: Distributed entanglement. '

           'Phys. Rev. A 61, 052306 (2000)'},



    'raw': 'Bakos, G., Noyes, R.W., Kovcs, G., Stanek, K.Z., Sasselov, D.D., '

           'Domsa, I. 2004, , 116, 266'},


The references provided are **not** all in the same style.



* **Reference 1 (Acosta et al.):**
    * **Style indicators:** Author last name, initials, collaboration name, journal abbreviation, volume, pages, year in parentheses. This looks like a common style for physics journals (e.g., APS style).
    * `authorD.Acosta etal. (collaborationCDF Collaboration), journalPhys. Rev. Lett. volume95, pages022003 (year2005).`

* **Reference 2 (Aldous):**
    * **Style indicators:** Author last name, initials, year in parentheses after author, title of work, "In" followed by edited volume title, pages, publisher. This is often seen in conference proceedings or edited books, sometimes resembling a variant of APA or Chicago.
    * `Aldous, D.J. (1985). Exchangeability and related topics. In cole d't de Probabilits de Saint-Flour XIII—1983, pages 1198. Springer.`

* **Reference 3 (Berg, Jordán):**
    * **Style indicators:** Author last name, initials (sometimes separated by a colon), title, "In:" followed by conference name/proceedings, LNCS (Lecture Notes in Computer Science) volume, page range, year in parentheses. This is very common in computer science conference proceedings.
    * `Berg, A.R., Jordn, T.: Algorithms for graph rigidity and scene analysis. In: Proc. 11th European Symposium on Algorithms (ESA '03), LNCS, vol. 2832, pp. 7889. (2003)`

* **Reference 4 (Aleksandrov and Strel'tsov):**
    * **Style indicators:** Initials first, then last name for authors, title of work, journal name (abbreviated), volume, year, page range, and then a separate entry for English translation. This particular style with the "English transi." separated out is less common but indicates a specific format.
    * `A. D. Aleksandrov and V. V. Strel'tsov, The isoperimetric problem and estimates of the length of a curve on a surface, Proc. Steklov Inst. Math. 76 (1965), 67-80; English transi., Proc. Steklov Inst Math. (1967), 81-99.`

* **Reference 5 (Coffman, Kundu, Wootters):**
    * **Style indicators:** Author last name, initials (sometimes with no comma or space between last name and initial), title, journal abbreviation, volume, page number, year in parentheses. Similar to the first one, likely a physics journal style.
    * `Coffman, V., Kundu, J., Wootters, W.K.: Distributed entanglement. Phys. Rev. A 61, 052306 (2000)`

* **Reference 6 (Bakos et al.):**
    * **Style indicators:** Last name, initials, then year, then some numbers that might correspond to volume and page (though the formatting is unusual with double commas and no explicit journal title). This looks incomplete or highly idiosyncratic.
    * `Bakos, G., Noyes, R.W., Kovcs, G., Stanek, K.Z., Sasselov, D.D., Domsa, I. 2004, , 116, 266`

**In summary:** There are variations in author-year placement, punctuation, abbreviations, and the inclusion of elements like "In:" or "Proc." which clearly indicate different citation styles are being used across these examples.

Extract --> Yes

Where do we go from here?

Classify + Recognize --> Not sure

Some failed cases:

1902.00005

\bibliographystyle{IEEEtran}
@ARTICLE{Clarke,
  author = {R. H. Clarke},
  title = {A statistical theory of mobile-radio reception},
  journal = {Bell Syst. Tech. J.},
  year = {1968},
  volume = {47},
  pages = {957-1000},
  publisher = {Bell Syst.}
}

@inproceedings{rezende_stochastic_2014,
	address = {Beijing, China},
	series = {{ICML}'14},
	title = {Stochastic {Backpropagation} and {Approximate} {Inference} in {Deep} {Generative} {Models}},
	url = {http://dl.acm.org/citation.cfm?id=3044805.3045035},
	abstract = {We},
	urldate = {2018-10-05},
	booktitle = {Proceedings of the 31st {International} {Conference} on {International} {Conference} on {Machine} {Learning} - {Volume} 32},
	publisher = {JMLR.org},
	author = {Rezende, Danilo Jimenez and Mohamed, Shakir and Wierstra, Daan},
	year = {2014},
	pages = {II--1278--II--1286}
}

@ARTICLE{Zemen,
  author = {T. Zemen and C. F. Mecklenbrauker and F. Kaltenberger and B. H. Fleury},
  title = {Minimum-Energy Band-Limited Predictor With Dynamic Subspace Selection for Time-Variant Flat-Fading Channels},
  journal = {IEEE Trans. on Signal Process.},
  year = {2007},
  volume = {55},
  pages = {4534-4548}
}


\bibitem

BibTeX-style references

The above has been resolved


1912.04801 -- Citation Section is missing

1912.04804 -- Weird formatting




For the next step, NER, LLM is only way that I could think of


Normal

```


\bibliographystyle{h-physrev-qian}
\begin{thebibliography}{10}


\bibitem{qcd-phase-fluctuations-02}
M.~A. Stephanov,
\newblock Phys. Rev. Lett. {\bf 102} (2009) 032301, arXiv:0809.3450.
```

Failed



```
\providecommand{\href}[2]{#2}\begin{thebibliography}{10}

\bibitem{strominger}
A.~Strominger, ``Superstrings with torsion,'' {\em Nucl. Phys.} {\bf B274}
  (1986)
253.


\bibitem{gauntlett-pakis}
J.~P. Gauntlett and S.~Pakis, ``{The geometry of $D=11$ Killing spinors},''
  {\em JHEP} {\bf 04} (2003) 039,
\href{http://arXiv.org/abs/hep-th/0212008}{{\tt hep-th/0212008}}.

\bibitem{ward}
R.~Ward, ``{Einstein--Weyl spaces and ${\rm SU}(\infty)$ Toda fields},'' {\em
  Classical and Quantum Gravity} {\bf 7} (1990), no.~4, L95.

\end{thebibliography}

\bibliographystyle{at}

```

The \href command is used to create a clickable link to an online resource, in this case, a pre-print on arXiv.org.


Some entries have extra lines between items (like 253. or the \href{} link).



--> is this good enough, requires more testing
\\bibitem(?:\[[^\]]*\])?\{([^\}]+)\}(.*?)(?=(\\bibitem|\Z))




